# ディープラーニングを適用した音楽検索システム (demo)

生成したが楽曲の特徴ベクトルを用いて画像検索を行います．

# 1. 楽曲データの準備

アノテーションのCSVファイルから、50個のタグを抽出します。
アノテーションのCSVにある楽曲ファイル名を参照し、mp3の楽曲データを読み込み、pythonのnumpy配列に変換します。

In [1]:
import numpy as np
from pydub import AudioSegment

def mp3_to_array(file):

    # MP3の読み込み
    song = AudioSegment.from_mp3(file)
    song_tmp = song._data

    # Numpy配列への変換
    song_nparr = np.frombuffer(song_tmp, np.int16)

    return song_nparr

In [2]:
%cd D:\dataset\demo\music_search
!pwd
!ffmpeg

D:\dataset\demo\music_search
/cygdrive/d/dataset/demo/music_search


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 9.2.1 (GCC) 20200122
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth --enable-libopenmpt
  libavutil      56. 31.100 / 56. 31.100
  li

In [3]:
import pandas as pd

# アノテーションの読み込み
tags_df = pd.read_csv('annotations_final.csv', delim_whitespace=True)
tags_df = tags_df.sample(frac=1)
#tags_df = tags_df[:3000]
tags_df = tags_df[:1000]

top50_tags = tags_df.iloc[:, 1:189].sum().sort_values(ascending=False).index[:50].tolist()
y =  tags_df[top50_tags].values

In [4]:
print(top50_tags)

['guitar', 'classical', 'slow', 'rock', 'strings', 'fast', 'techno', 'drums', 'electronic', 'piano', 'violin', 'ambient', 'beat', 'synth', 'vocal', 'male', 'male vocal', 'indian', 'female', 'pop', 'loud', 'opera', 'singing', 'quiet', 'solo', 'harpsichord', 'no vocal', 'sitar', 'soft', 'vocals', 'flute', 'no vocals', 'voice', 'woman', 'man', 'country', 'cello', 'classic', 'male voice', 'harp', 'choir', 'no voice', 'violins', 'dance', 'male vocals', 'modern', 'female vocal', 'beats', 'new age', 'weird']


In [5]:
AudioSegment.from_mp3('D:\\dataset\\demo\\music_search\\3\\sherefe-opium-05-sokol_mi_leta-175-204.mp3')

In [7]:
import os

myfiles = tags_df.mp3_path.values

path = 'D:/dataset/demo/music_search/'

AudioSegment.ffmpeg = "C:\\Program Files\\ffmpeg-4.2.2-win64-static\\bin\\ffmpeg.exe" 

# Error on Windows
AudioSegment.from_mp3('D:\\dataset\\demo\\music_search\\3\\sherefe-opium-05-sokol_mi_leta-175-204.mp3')
#AudioSegment.from_mp3('D:\\dataset\\demo\\music_search\\3\\american_baroque-mozart_4_quartets_for_strings_and_winds-01-oboe_quartet_in_f_k370__allegro-0-29.mp3')


X = np.array([ mp3_to_array(os.path.join(path, file)) for file in myfiles ])
  
X = X.reshape(X.shape[0], X.shape[1], 1)

CouldntDecodeError: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 9.2.1 (GCC) 20200122
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec -

# 2. ディープラーニングの学習



In [ ]:
# TrainとTest用のデータに分割

from sklearn.model_selection import train_test_split
random_state = 42

#train_X, test_X, train_y, test_y = train_test_split(X, y[0:997], test_size=0.2, random_state=random_state)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=random_state)

In [ ]:
print (train_X[0])

In [ ]:
# 複数ラベル用の損失関数，評価関数の定義

import keras.backend as K

# 損失関数
def categorical_loss(y_true, y_pred):
    return K.categorical_crossentropy(y_true, y_pred)

def binary_loss(y_true, y_pred):
    bce = K.binary_crossentropy(y_true, y_pred)
    return K.sum(bce, axis=-1)

# 評価関数
def total_acc(y_true, y_pred):
    pred = K.cast(K.greater_equal(y_pred, 0.5), "float")
    flag = K.cast(K.equal(y_true, pred), "float")
    return K.prod(flag, axis=-1)

def binary_acc(y_true, y_pred):
    pred = K.cast(K.greater_equal(y_pred, 0.5), "float")
    flag = K.cast(K.equal(y_true, pred), "float")
    return K.mean(flag, axis=-1)

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense,  Flatten, Input
from keras.layers import Conv1D, MaxPooling1D

features = train_X.shape[1]

x_inputs = Input(shape=(features, 1), name='x_inputs') # (特徴量数, チャネル数)
x = Conv1D(128, 256, strides=256,
           padding='valid', activation='relu') (x_inputs)
x = Conv1D(32, 8, activation='relu') (x) # (チャネル数, フィルタの長さ )
x = MaxPooling1D(4) (x) # （フィルタの長さ）
x = Conv1D(32, 8, activation='relu') (x)
x = MaxPooling1D(4) (x)
x = Conv1D(32, 8, activation='relu') (x)
x = MaxPooling1D(4) (x)
x = Conv1D(32, 8, activation='relu') (x)
x = MaxPooling1D(4) (x)
x = Flatten() (x)
x = Dense(100, activation='relu') (x) #（ユニット数）
x_outputs = Dense(50, activation='sigmoid', name='x_outputs') (x)

model = Model(inputs=x_inputs, outputs=x_outputs)
model.compile(optimizer='adam',
#              loss='categorical_crossentropy',
#              metrics=['accuracy'])
              loss=binary_loss,
              metrics=[total_acc, binary_acc])


In [ ]:
#model.fit(train_X, train_y, batch_size=600, epochs=50)
model.fit(train_X, train_y, batch_size=600, epochs=25)

In [ ]:
model.save('musci_search_dl.h5')

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense,  Flatten, Input
from keras.layers import Conv1D, MaxPooling1D

from keras.models import load_model

model = load_model('musci_search_dl.h5', custom_objects={'binary_loss': binary_loss, 'total_acc': total_acc,  'binary_acc': binary_acc})

In [ ]:
from sklearn.metrics import roc_auc_score
pred_y_x = model.predict(test_X, batch_size=50)
#print(roc_auc_score(test_y, pred_y_x)) # => 0.668582599155
print(pred_y_x)

# 3. 楽曲の特徴抽出

ディープラーニングにより学習したニューラルネットワークを用いて、楽曲の特徴抽出をする。

In [ ]:
intermed_model = Model(inputs=model.input, outputs=model.get_layer('dense_1').output)
intermed_model.summary()

In [ ]:
class CBIR(object):
    def __init__(self):
        pass

    def train(self, X):
        """ X is N x D where each row is an example. Y is 1-dimension of size N """
        # the nearest neighbor classifier simply remembers all the training data
        self.X = X
    
    def search(self, q, k):
        distances = np.sum(np.abs(self.X - q), axis = 1)
        
        sorted_indexes = np.argsort(distances)
        #max_k = sorted_indexes[-(k+1):-1]
        min_k = sorted_indexes[:k+1]
        
        #return max_k
        return min_k

In [ ]:
targets = intermed_model.predict(X)
#print(targets[0:2])

# 4. 楽曲検索

In [ ]:
import IPython.display

q_idx = 456 # 4, 456, 
q = targets[q_idx,:]
print(q)
cbir = CBIR() # create a Nearest Neighbor classifier class
cbir.train(targets)
result = cbir.search(q, 10) # predict music

IPython.display.display(IPython.display.Audio(myfiles[q_idx]))
print(result)

In [ ]:
print(result)
for id in result:
  print(id)
  IPython.display.display(IPython.display.Audio(myfiles[id]))

In [ ]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))